<a href="https://colab.research.google.com/github/felolivee/DVA-NYC_Congestion/blob/Ashley/DVA_Project_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download from Kaggle
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d jeffsinsel/nyc-fhvhv-data

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/jeffsinsel/nyc-fhvhv-data
License(s): CC0-1.0
100% 17.8G/17.8G [02:56<00:00, 56.1MB/s]
100% 17.8G/17.8G [02:57<00:00, 108MB/s] 


In [2]:
#unzip files in nyc-fhvhv-data from Kaggle
! unzip nyc-fhvhv-data.zip

Archive:  nyc-fhvhv-data.zip
  inflating: data_dictionary_trip_records_hvfhs.pdf  
  inflating: fhvhv_tripdata_2019-02.parquet  
  inflating: fhvhv_tripdata_2019-03.parquet  
  inflating: fhvhv_tripdata_2019-04.parquet  
  inflating: fhvhv_tripdata_2019-05.parquet  
  inflating: fhvhv_tripdata_2019-06.parquet  
  inflating: fhvhv_tripdata_2019-07.parquet  
  inflating: fhvhv_tripdata_2019-08.parquet  
  inflating: fhvhv_tripdata_2019-09.parquet  
  inflating: fhvhv_tripdata_2019-10.parquet  
  inflating: fhvhv_tripdata_2019-11.parquet  
  inflating: fhvhv_tripdata_2019-12.parquet  
  inflating: fhvhv_tripdata_2020-01.parquet  
  inflating: fhvhv_tripdata_2020-02.parquet  
  inflating: fhvhv_tripdata_2020-03.parquet  
  inflating: fhvhv_tripdata_2020-04.parquet  
  inflating: fhvhv_tripdata_2020-05.parquet  
  inflating: fhvhv_tripdata_2020-06.parquet  
  inflating: fhvhv_tripdata_2020-07.parquet  
  inflating: fhvhv_tripdata_2020-08.parquet  
  inflating: fhvhv_tripdata_2020-09.parquet

In [14]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import glob

# initialize spark
spark = SparkSession.builder.appName("NYC_Rides").getOrCreate()

# list of all parquet files from content folder
parquet_files = glob.glob('/content/*.parquet')

removed_summary = {}
combined_df = None

for file in parquet_files:
    # read parquet files get features
    df = spark.read.parquet(file).select("Hvfhs_license_num", "Pickup_datetime", "DropOff_datetime", "PULocationID", "DOLocationID", "base_passenger_fare", "trip_miles", "tips", "driver_pay", "trip_time")
    initial_count = df.count()

    # clean data
    df_clean = df.filter(
        (F.col("Pickup_datetime").isNotNull()) &
        (F.col("DropOff_datetime").isNotNull()) &
        (F.col("PULocationID").isNotNull()) &
        (F.col("DOLocationID").isNotNull()) &
        (F.col("base_passenger_fare").isNotNull()) &
        (F.col("trip_miles").isNotNull()) &
        (F.col("tips").isNotNull()) &
        (F.col("driver_pay").isNotNull()) &
        (F.col("trip_time").isNotNull()) &
        (F.col("Hvfhs_license_num").isNotNull()) &
        (F.col("base_passenger_fare") > 0) &
        (F.col("trip_miles") > 0) &
        (F.col("driver_pay") > 0) &
        (F.col("trip_time") > 0)
    )
    clean_count = df_clean.count()
    removed_summary[file] = initial_count - clean_count

    # combine clean DFs into one big DF
    if combined_df is None:
        combined_df = df_clean
    else:
        combined_df = combined_df.union(df_clean)

# check how many of which file were removed
print("Removal Summary:", removed_summary)

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# calculate how many total rows of data were removed
tot = 0
for key,val in removed_summary.items():
  tot += int(val)

count = combined_df.count()
percent_removed = tot/count * 100

print("{}% was removed from a total of {} rides in 46 months".format(percent_removed, count))

5.159562385472075% was removed from a total of 708720164 rides in 46 months


In [ ]:
# show schema
combined_df.printSchema()

root
 |-- Hvfhs_license_num: string (nullable = true)
 |-- Pickup_datetime: timestamp_ntz (nullable = true)
 |-- DropOff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- trip_time: long (nullable = true)



In [9]:
# aggregate features for Tableau file

add_hour = combined_df.withColumn("pickup_hour", F.date_format(F.col("Pickup_datetime"), "k")).withColumn("dropoff_hour", F.date_format(F.col("DropOff_datetime"), "k"))

pickups = add_hour.groupBy(["PULocationID", "pickup_hour", "Hvfhs_license_num"]).agg(F.count("PULocationID").alias("PU_count"),F.mean("trip_miles").alias("avg_trip_miles_PU"), F.mean("driver_pay").alias("avg_driver_pay_PU"))

dropoffs = add_hour.groupBy(["DOLocationID", "dropoff_hour", "Hvfhs_license_num"]).agg(F.count("DOLocationID").alias("DO_count"),F.mean("trip_miles").alias("avg_trip_miles_DO"), F.mean("driver_pay").alias("avg_driver_pay_DO"))

dropoffs=dropoffs.withColumnRenamed("Hvfhs_license_num", "Hvfhs_license_num_2")

tableau_df = pickups.join(dropoffs, (pickups.pickup_hour == dropoffs.dropoff_hour) & (pickups.PULocationID == dropoffs.DOLocationID)&(pickups.Hvfhs_license_num == dropoffs.Hvfhs_license_num_2), "fullouter")

tableau_df = tableau_df.withColumn("rideType", F.when(F.col("Hvfhs_license_num") == "HV0002", "Juno")
     .when(F.col("Hvfhs_license_num") == "HV0003", "Uber")
     .when(F.col("Hvfhs_license_num") == "HV0004", "Via")
     .when(F.col("Hvfhs_license_num") == "HV0005", "Lyft"))


In [10]:
tableau_df.show()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
tableau_df.write.option('header', True).csv('dva_project.csv')

Py4JJavaError: An error occurred while calling o3254.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.wai